<a href="https://colab.research.google.com/github/aaronverive/Coursera_Capstone/blob/master/Battle_of_Neighborhoods_Wk5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#import beautiful soup
from urllib.request import urlopen
from bs4 import BeautifulSoup


print('Libraries imported.')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Libraries imported.


### **Geographic Data for Lafayette**

In [2]:
address = 'Lafayette,louisiana'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lafayette are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Lafayette are 30.2240897, -92.0198427.


### **Foursquare API Usage**

In [3]:
CLIENT_ID = "4VFSWEDAWJR41KSDTKQVT3QN05HYO5Z20BJTSGXT2BTCWM0U"  # your Foursquare ID
CLIENT_SECRET = "FEJI3IAK1NXILARNDWBNMDH4A2SYR55HRHT2SMUY4SJNTQPM" # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4VFSWEDAWJR41KSDTKQVT3QN05HYO5Z20BJTSGXT2BTCWM0U
CLIENT_SECRET:FEJI3IAK1NXILARNDWBNMDH4A2SYR55HRHT2SMUY4SJNTQPM


In [4]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=4VFSWEDAWJR41KSDTKQVT3QN05HYO5Z20BJTSGXT2BTCWM0U&client_secret=FEJI3IAK1NXILARNDWBNMDH4A2SYR55HRHT2SMUY4SJNTQPM&v=20180605&ll=30.2240897,-92.0198427&radius=500&limit=100'

In [0]:
results = requests.get(url).json()
#results

In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
#pull the actual data from the Foursquare API

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.id', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues

#column formatting
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,id,categories,lat,lng
0,Pamplona Tapas Bar,4b645082f964a5201aaa2ae3,Tapas Restaurant,30.223133,-92.018842
1,The French Press,4b4e1609f964a52008e126e3,Breakfast Spot,30.224067,-92.017341
2,Pop's Poboys,554505d3498eb39976b193b7,Sandwich Place,30.221788,-92.018981
3,Festival International De Louisiane,4bb3a9f035f0c9b6cfb5bc83,General Entertainment,30.224735,-92.018856
4,Carpe Diem! Gelato - Espresso Bar,4e0e6402fa76d62f4448b231,Dessert Shop,30.221663,-92.018885
5,Acadiana Center for the Arts,4b87094cf964a520ceac31e3,Theater,30.223930,-92.019100
6,The Wurst Biergarten,57476694498eca2fab9a2093,Beer Bar,30.224757,-92.018692
7,Jefferson Street Pub,4b74d607f964a520a0f42de3,Bar,30.225770,-92.018780
8,Tsunami,4b48e8dcf964a520f45c26e3,Sushi Restaurant,30.226454,-92.017849
9,Parc International,4bad1814f964a520f12c3be3,Music Venue,30.225448,-92.016770


In [8]:
# find a list of unique categories from the API so we can see what may or may not fit for restaurants

nearby_venues['categories'].unique()

array(['Tapas Restaurant', 'Breakfast Spot', 'Sandwich Place',
       'General Entertainment', 'Dessert Shop', 'Theater', 'Beer Bar',
       'Bar', 'Sushi Restaurant', 'Music Venue', 'Other Great Outdoors',
       'Cajun / Creole Restaurant', 'Café', 'Science Museum',
       'Seafood Restaurant', 'Mexican Restaurant', 'Taco Place', 'Hotel',
       'Museum', 'Hookah Bar', 'Sports Bar', 'Performing Arts Venue',
       'Bank', 'Pharmacy', 'Pizza Place', 'Sculpture Garden',
       'Clothing Store', 'Health & Beauty Service', 'Record Shop'],
      dtype=object)

In [9]:
# creating a list of categorie to remove from our dataframe because they are not restaurants
# I am sure there is a function that can be written to do this at scale but since it was a small list, I did it manually

removal_list = ['Gym / Fitness Center', 'Bakery', 'Park', "Women's Store", 'Sporting Goods Shop', 'Dog Run', 'Gaming Cafe',
               'Optical Shop', 'Yoga Studio', 'Pet Store', 'Shoe Repair', 'Jewelry Store', 'Record Shop', 'Juice Bar', 
               'Cosmetics Shop', 'Business Service', 'Salon / Barbershop', 'Liquor Store', 'Grocery Store', 'Stationery Store',
               'Pilates Studio', 'Dessert Shop', 'Bookstore', 'Concert Hall', 'Video Game Store', 'Pharmacy', 'Mobile Phone Shop',
               'Deli / Bodega']

nearby_venues2 = nearby_venues.copy()


#getting a clear dataframe of just restaurants
nearby_venues2 = nearby_venues2[~nearby_venues2['categories'].isin(removal_list)]
nearby_venues2

,name,id,categories,lat,lng
0,Pamplona Tapas Bar,4b645082f964a5201aaa2ae3,Tapas Restaurant,30.223133,-92.018842
1,The French Press,4b4e1609f964a52008e126e3,Breakfast Spot,30.224067,-92.017341
2,Pop's Poboys,554505d3498eb39976b193b7,Sandwich Place,30.221788,-92.018981
3,Festival International De Louisiane,4bb3a9f035f0c9b6cfb5bc83,General Entertainment,30.224735,-92.018856
5,Acadiana Center for the Arts,4b87094cf964a520ceac31e3,Theater,30.223930,-92.019100
6,The Wurst Biergarten,57476694498eca2fab9a2093,Beer Bar,30.224757,-92.018692
7,Jefferson Street Pub,4b74d607f964a520a0f42de3,Bar,30.225770,-92.018780
8,Tsunami,4b48e8dcf964a520f45c26e3,Sushi Restaurant,30.226454,-92.017849
9,Parc International,4bad1814f964a520f12c3be3,Music Venue,30.225448,-92.016770
10,Antlers Downtown,4b4b60def964a520609926e3,Bar,30.224307,-92.018620


In [10]:
#let's get a list of venues

venue_id_list = nearby_venues2['id'].tolist()
venue_id_list

['4b645082f964a5201aaa2ae3',
 '4b4e1609f964a52008e126e3',
 '554505d3498eb39976b193b7',
 '4bb3a9f035f0c9b6cfb5bc83',
 '4b87094cf964a520ceac31e3',
 '57476694498eca2fab9a2093',
 '4b74d607f964a520a0f42de3',
 '4b48e8dcf964a520f45c26e3',
 '4bad1814f964a520f12c3be3',
 '4b4b60def964a520609926e3',
 '4c00628bad46c9b6f9708f66',
 '4b48c08ef964a520645526e3',
 '4b38b365f964a520874f25e3',
 '4c62227c4b5176b09a910017',
 '4c3100ad452620a1f5fb1f0f',
 '4babf246f964a52002d83ae3',
 '4b48e5ebf964a520575c26e3',
 '526407ae11d2566dc09f69da',
 '4e54e7608130102d7e60fdad',
 '4b48d768f964a520865926e3',
 '4b48e81af964a520cc5c26e3',
 '4b7458eaf964a520c8d62de3',
 '4af3fdc5f964a520c5ef21e3',
 '4bd0947ab221c9b6b7f9d3d0',
 '5179d13de4b0b5d316a91960',
 '4b2d289cf964a520c9cf24e3',
 '4bdc9522c79cc928693987e9',
 '4dcfc7f8183899ddfaf48427',
 '4ba4137ff964a520e77e38e3',
 '4b4cf6fef964a520c2c526e3',
 '4bcf96c577b29c7483738982',
 '4e2eee69b61c88c3a7929e26',
 '4bb80a717421a5932876c140',
 '4b47cf3cf964a520b33f26e3',
 '4bd32a7fa8b3

In [11]:

#set up to pull the likes from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in venue_id_list:
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    like_list.append(likes)
print(like_list)

[42, 86, 21, 65, 16, 13, 18, 34, 19, 9, 8, 46, 32, 3, 14, 24, 33, 17, 8, 7, 31, 4, 34, 8, 14, 45, 7, 4, 10, 13, 8, 0, 7, 4, 5, 0, 7, 1, 1, 0, 2, 0, 4]


In [12]:
#double check that we did not lose any venues based on if likes were available

print(len(like_list))
print(len(venue_id_list))

43
43


### **Data Preparation**

In [13]:
#let's make a copy of our initial dataframe just in case anything goes wrong

to_venues = nearby_venues2.copy()
to_venues.head()

,name,id,categories,lat,lng
0,Pamplona Tapas Bar,4b645082f964a5201aaa2ae3,Tapas Restaurant,30.223133,-92.018842
1,The French Press,4b4e1609f964a52008e126e3,Breakfast Spot,30.224067,-92.017341
2,Pop's Poboys,554505d3498eb39976b193b7,Sandwich Place,30.221788,-92.018981
3,Festival International De Louisiane,4bb3a9f035f0c9b6cfb5bc83,General Entertainment,30.224735,-92.018856
5,Acadiana Center for the Arts,4b87094cf964a520ceac31e3,Theater,30.223930,-92.019100


In [14]:
# add in the list of likes

to_venues['total likes'] = like_list
to_venues.head()

,name,id,categories,lat,lng,total likes
0,Pamplona Tapas Bar,4b645082f964a5201aaa2ae3,Tapas Restaurant,30.223133,-92.018842,42
1,The French Press,4b4e1609f964a52008e126e3,Breakfast Spot,30.224067,-92.017341,86
2,Pop's Poboys,554505d3498eb39976b193b7,Sandwich Place,30.221788,-92.018981,21
3,Festival International De Louisiane,4bb3a9f035f0c9b6cfb5bc83,General Entertainment,30.224735,-92.018856,65
5,Acadiana Center for the Arts,4b87094cf964a520ceac31e3,Theater,30.223930,-92.019100,16


In [15]:
# now let's bin total likes

print(to_venues['total likes'].max())
print(to_venues['total likes'].min())
print(to_venues['total likes'].median())
print(to_venues['total likes'].mean())

86
0
9.0
16.837209302325583


In [16]:
# let's visualize our total likes based on a histogram

import matplotlib.pyplot as plt
to_venues['total likes'].hist(bins=4)
plt.show()

In [17]:
# what are the bins we want to use?

print(np.percentile(to_venues['total likes'], 25))
print(np.percentile(to_venues['total likes'], 50))
print(np.percentile(to_venues['total likes'], 75))

4.0
9.0
22.5


In [0]:

# now we have our bin values so let's set them to the appropriate values
# less than 24, 24-45, 45-76, 76>
# poor, below avg, abv avg, great

poor = to_venues['total likes']<=24
below_avg = to_venues[(to_venues['total likes']>24) & (to_venues['total likes']<=45)]
abv_avg = to_venues[(to_venues['total likes']>45) & (to_venues['total likes']<=76)]
great = to_venues['total likes']>76

In [0]:
# let's set up a function that will re-categorize our restaurants based on likes

def conditions(s):
    if s['total likes']<=24:
        return 'poor'
    if s['total likes']<=45:
        return 'below avg'
    if s['total likes']<=76:
        return 'avg avg'
    if s['total likes']>76:
        return 'great'

to_venues['total likes_cat']=to_venues.apply(conditions, axis=1)


In [20]:
to_venues

,name,id,categories,lat,lng,total likes,total likes_cat
0,Pamplona Tapas Bar,4b645082f964a5201aaa2ae3,Tapas Restaurant,30.223133,-92.018842,42,below avg
1,The French Press,4b4e1609f964a52008e126e3,Breakfast Spot,30.224067,-92.017341,86,great
2,Pop's Poboys,554505d3498eb39976b193b7,Sandwich Place,30.221788,-92.018981,21,poor
3,Festival International De Louisiane,4bb3a9f035f0c9b6cfb5bc83,General Entertainment,30.224735,-92.018856,65,avg avg
5,Acadiana Center for the Arts,4b87094cf964a520ceac31e3,Theater,30.223930,-92.019100,16,poor
6,The Wurst Biergarten,57476694498eca2fab9a2093,Beer Bar,30.224757,-92.018692,13,poor
7,Jefferson Street Pub,4b74d607f964a520a0f42de3,Bar,30.225770,-92.018780,18,poor
8,Tsunami,4b48e8dcf964a520f45c26e3,Sushi Restaurant,30.226454,-92.017849,34,below avg
9,Parc International,4bad1814f964a520f12c3be3,Music Venue,30.225448,-92.016770,19,poor
10,Antlers Downtown,4b4b60def964a520609926e3,Bar,30.224307,-92.018620,9,poor


In [44]:
# let's star the process for re-categorizing the categories

to_venues['categories'].unique()

array(['Tapas Restaurant', 'Breakfast Spot', 'Sandwich Place',
       'General Entertainment', 'Theater', 'Beer Bar', 'Bar',
       'Sushi Restaurant', 'Music Venue', 'Other Great Outdoors',
       'Cajun / Creole Restaurant', 'Café', 'Science Museum',
       'Seafood Restaurant', 'Mexican Restaurant', 'Taco Place', 'Hotel',
       'Museum', 'Hookah Bar', 'Sports Bar', 'Performing Arts Venue',
       'Bank', 'Pizza Place', 'Sculpture Garden', 'Clothing Store',
       'Health & Beauty Service'], dtype=object)

In [0]:
# let's create our new categories and create a function to apply those to our existing data


bars = ['Pub', 'Cocktail Bar', 'Bar', 'Dive Bar', 'Sports Bar']
other = ['Bagel Shop', 'Tea Room', 'Donut Shop', 'Coffee Shop', 'Bubble Tea Shop', 'Sandwich Place', 'Boutique', 'Ice Cream Shop']
euro_asia_indian_food = ['Falafel Restaurant', 'Korean Restaurant','Sushi Restaurant', 'Indian Restaurant', 'Japanese Restaurant', 'Poke Place', 'Thai Restaurant', 'Vietnamese Restaurant']
mex_southam_food = ['Cuban Restaurant', 'Mexican Restaurant', 'South American Restaurant', 'Latin American Restaurant']
american_food = ['Burger Joint', 'Restaurant', 'American Restaurant']
italian_food = ['Italian Restaurant', 'Seafood Restaurant', 'Pizza Place']

def conditions2(s):
    if s['categories'] in bars:
        return 'bars'
    if s['categories'] in other:
        return 'other'
    if s['categories'] in euro_asia_indian_food:
        return 'euro asia indian food'
    if s['categories'] in mex_southam_food:
        return 'mex southam food'
    if s['categories'] in american_food:
        return 'american food'
    if s['categories'] in italian_food:
        return 'italian food'

to_venues['categories_new']=to_venues.apply(conditions2, axis=1)

In [46]:
to_venues

,name,id,categories,lat,lng,total likes,total likes_cat,categories_new,label
0,Pamplona Tapas Bar,4b645082f964a5201aaa2ae3,Tapas Restaurant,30.223133,-92.018842,42,below avg,None,1
1,The French Press,4b4e1609f964a52008e126e3,Breakfast Spot,30.224067,-92.017341,86,great,None,0
2,Pop's Poboys,554505d3498eb39976b193b7,Sandwich Place,30.221788,-92.018981,21,poor,other,0
3,Festival International De Louisiane,4bb3a9f035f0c9b6cfb5bc83,General Entertainment,30.224735,-92.018856,65,avg avg,None,2
5,Acadiana Center for the Arts,4b87094cf964a520ceac31e3,Theater,30.223930,-92.019100,16,poor,None,0
6,The Wurst Biergarten,57476694498eca2fab9a2093,Beer Bar,30.224757,-92.018692,13,poor,None,0
7,Jefferson Street Pub,4b74d607f964a520a0f42de3,Bar,30.225770,-92.018780,18,poor,bars,3
8,Tsunami,4b48e8dcf964a520f45c26e3,Sushi Restaurant,30.226454,-92.017849,34,below avg,euro asia indian food,1
9,Parc International,4bad1814f964a520f12c3be3,Music Venue,30.225448,-92.016770,19,poor,None,0
10,Antlers Downtown,4b4b60def964a520609926e3,Bar,30.224307,-92.018620,9,poor,bars,3


In [47]:
# one hot encoding
to_onehot = pd.get_dummies(to_venues[['categories_new', 'total likes_cat']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Name'] = to_venues['name'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

to_onehot.head()

,Name,bars,euro asia indian food,italian food,mex southam food,other,avg avg,below avg,great,poor
0,Pamplona Tapas Bar,0,0,0,0,0,0,1,0,0
1,The French Press,0,0,0,0,0,0,0,1,0
2,Pop's Poboys,0,0,0,0,1,0,0,0,1
3,Festival International De Louisiane,0,0,0,0,0,1,0,0,0
5,Acadiana Center for the Arts,0,0,0,0,0,0,0,0,1


In [48]:

cluster_df = to_onehot.drop('Name', axis=1)

k_clusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(cluster_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 2, 0, 0, 3, 1, 0, 3], dtype=int32)

In [49]:
to_venues['label'] = kmeans.labels_
to_venues.head()

,name,id,categories,lat,lng,total likes,total likes_cat,categories_new,label
0,Pamplona Tapas Bar,4b645082f964a5201aaa2ae3,Tapas Restaurant,30.223133,-92.018842,42,below avg,None,1
1,The French Press,4b4e1609f964a52008e126e3,Breakfast Spot,30.224067,-92.017341,86,great,None,0
2,Pop's Poboys,554505d3498eb39976b193b7,Sandwich Place,30.221788,-92.018981,21,poor,other,0
3,Festival International De Louisiane,4bb3a9f035f0c9b6cfb5bc83,General Entertainment,30.224735,-92.018856,65,avg avg,None,2
5,Acadiana Center for the Arts,4b87094cf964a520ceac31e3,Theater,30.223930,-92.019100,16,poor,None,0


In [42]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i+x+(i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_venues['lat'], to_venues['lng'], to_venues['name'], to_venues['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1

In [43]:
to_venues.loc[to_venues['label']==0]

,name,id,categories,lat,lng,total likes,total likes_cat,categories_new,label
1,The French Press,4b4e1609f964a52008e126e3,Breakfast Spot,30.224067,-92.017341,86,great,None,0
2,Pop's Poboys,554505d3498eb39976b193b7,Sandwich Place,30.221788,-92.018981,21,poor,other,0
5,Acadiana Center for the Arts,4b87094cf964a520ceac31e3,Theater,30.223930,-92.019100,16,poor,None,0
6,The Wurst Biergarten,57476694498eca2fab9a2093,Beer Bar,30.224757,-92.018692,13,poor,None,0
9,Parc International,4bad1814f964a520f12c3be3,Music Venue,30.225448,-92.016770,19,poor,None,0
11,Parc San Souci,4c00628bad46c9b6f9708f66,Other Great Outdoors,30.224517,-92.017597,8,poor,None,0
14,Subway,4c62227c4b5176b09a910017,Sandwich Place,30.225412,-92.018675,3,poor,other,0
15,Lafayette Science Museum,4c3100ad452620a1f5fb1f0f,Science Museum,30.225641,-92.018215,14,poor,None,0
16,Don's Seafood & Steakhouse,4babf246f964a52002d83ae3,Seafood Restaurant,30.224244,-92.016631,24,poor,italian food,0
18,Black Café,526407ae11d2566dc09f69da,Café,30.228073,-92.018814,17,poor,None,0


Cluster 2

In [29]:
to_venues.loc[to_venues['label']==1]

,name,id,categories,lat,lng,total likes,total likes_cat,categories_new,label
0,Pamplona Tapas Bar,4b645082f964a5201aaa2ae3,Tapas Restaurant,30.223133,-92.018842,42,below avg,None,1
8,Tsunami,4b48e8dcf964a520f45c26e3,Sushi Restaurant,30.226454,-92.017849,34,below avg,euro asia indian food,1
13,Dwyer's Cafe,4b38b365f964a520874f25e3,Café,30.226655,-92.017300,32,below avg,None,1
17,Agave Mexican Cantina & Grill,4b48e5ebf964a520575c26e3,Mexican Restaurant,30.224000,-92.017570,33,below avg,mex southam food,1
21,Taco Sisters,4b48e81af964a520cc5c26e3,Taco Place,30.223280,-92.014986,31,below avg,None,1
23,The Green Room,4af3fdc5f964a520c5ef21e3,Bar,30.226871,-92.016866,34,below avg,bars,1
26,Artmosphere,4b2d289cf964a520c9cf24e3,Hookah Bar,30.220519,-92.017200,45,below avg,None,1


Cluster 3

In [30]:
to_venues.loc[to_venues['label']==2]

,name,id,categories,lat,lng,total likes,total likes_cat,categories_new,label
3,Festival International De Louisiane,4bb3a9f035f0c9b6cfb5bc83,General Entertainment,30.224735,-92.018856,65,avg avg,None,2
12,Johnson's Boucaniere,4b48c08ef964a520645526e3,Cajun / Creole Restaurant,30.221221,-92.022751,46,avg avg,None,2


Cluster 4

In [31]:

to_venues.loc[to_venues['label']==3]

,name,id,categories,lat,lng,total likes,total likes_cat,categories_new,label
7,Jefferson Street Pub,4b74d607f964a520a0f42de3,Bar,30.225770,-92.018780,18,poor,bars,3
10,Antlers Downtown,4b4b60def964a520609926e3,Bar,30.224307,-92.018620,9,poor,bars,3
20,NiteTown,4b48d768f964a520865926e3,Bar,30.225197,-92.018848,7,poor,bars,3
27,Legends Annex,4bdc9522c79cc928693987e9,Sports Bar,30.225961,-92.017247,7,poor,bars,3
30,Marley's,4b4cf6fef964a520c2c526e3,Bar,30.226199,-92.017435,13,poor,bars,3
33,Legends Downtown,4bb80a717421a5932876c140,Bar,30.226183,-92.017517,7,poor,bars,3
34,The Office,4b47cf3cf964a520b33f26e3,Bar,30.226747,-92.016773,4,poor,bars,3
38,Arcadian Bar and Grill,52731805498ed32cc0e1f5cb,Bar,30.226878,-92.016997,7,poor,bars,3


In [0]:
sorted_venues = to_venues.sort_values(['total likes'],ascending=False)

In [52]:
sorted_venues.head(3)

,name,id,categories,lat,lng,total likes,total likes_cat,categories_new,label
1,The French Press,4b4e1609f964a52008e126e3,Breakfast Spot,30.224067,-92.017341,86,great,None,0
3,Festival International De Louisiane,4bb3a9f035f0c9b6cfb5bc83,General Entertainment,30.224735,-92.018856,65,avg avg,None,2
12,Johnson's Boucaniere,4b48c08ef964a520645526e3,Cajun / Creole Restaurant,30.221221,-92.022751,46,avg avg,None,2


### **Results/Conclusion**

Given the various restaurants in Lafayette, the best restaurant to eat would be "The French Press". There are other viable options nearby such as "Festival International De Louisiane" and "Johnson's Boucaniere" worth eating at. 